In [ ]:
import numpy as np
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras.models import load_model
from keras import regularizers, optimizers
from keras.initializers import glorot_normal, RandomNormal

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)# .reshape((y_train.shape[0], 10,1))
y_test = np_utils.to_categorical(y_test,num_classes)# .reshape((y_test.shape[0], 10,1))


In [ ]:
# data augmentation
datagen = ImageDataGenerator(
    fill_mode="constant",
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True
    )
datagen.fit(x_train)

In [ ]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

def swish_125(x):
    return 1.25*x*K.sigmoid(x)

def schedule(x):
    if x < 60:
        return 0.1
    elif x < 120:
        return 0.1*0.2
    elif x < 160:
        return 0.1*0.2*0.2
    else:
        return 0.1*0.2*0.2*0.2

In [ ]:
def create_model(act, act_name):
    nn = {"act": act, "act_name": act_name}
    
    weight_decay = 5e-3
    s = 2
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=glorot_normal(), input_shape=x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    
    # Block 2
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 3
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 4
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(act))
    
    # First Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    
    # Block 5
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 6
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 7
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    # Second Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    
    # Block 8
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 9
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    # Third Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    
    # Block 10
    model.add(Conv2D(512, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    # Block 11  
    model.add(Conv2D(2048, (1,1), padding='same', kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 12  
    model.add(Conv2D(256, (1,1), padding='same', kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fourth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))


    # Block 13
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fifth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    # Final Classifier
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    
    nn["model"] = model
    return nn

In [ ]:
nn = create_model("relu", "relu")
print("\n", nn) # Ensure everything's ok
nn["model"].summary()
model = nn["model"]
batch_size = 128

batch_size  = 128
epochs = 200

opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=True)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
nn["model"].compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = nn["model"].fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
print(his.history)

In [ ]:
nn = create_model("relu", "relu")
print("\n", nn) # Ensure everything's ok
nn["model"].summary()
model = nn["model"]
batch_size = 128

batch_size  = 128
epochs = 200

opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=True)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
nn["model"].compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = nn["model"].fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
print(his.history)
     

In [ ]:
nn = create_model(swish, "swish")
print("\n", nn) # Ensure everything's ok
nn["model"].summary()
model = nn["model"]
batch_size = 128
epochs = 200

opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=True)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
nn["model"].compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = nn["model"].fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
print(his.history)

In [ ]:
nn = create_model(swish_125, "swish_150")
print("\n", nn) # Ensure everything's ok
nn["model"].summary()
model = nn["model"]
batch_size = 128
epochs = 150

opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=True)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
nn["model"].compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = nn["model"].fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
print(his.history)

In [ ]:
def swish_1125(x):
    return 1.125*x*K.sigmoid(x)

nn = create_model(swish_1125, "swish_150")
print("\n", nn) # Ensure everything's ok
nn["model"].summary()
model = nn["model"]
batch_size = 128
epochs = 150

opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=True)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
nn["model"].compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = nn["model"].fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
print(his.history)